# 第5章: 出力の書式設定とClaude の代弁

- [レッスン](#lesson)
- [演習](#exercises)
- [サンプル練習場](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を設定してください。

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# New argument added for prefill text, with a default value of an empty string
def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## レッスン

**Claude は様々な方法で出力をフォーマットできます**。そうするよう求めるだけです！

その方法の一つは、XMLタグを使用して、他の余分なテキストから応答を分離することです。あなたはすでに、XMLタグを使ってプロンプトをより明確でClaude が解析しやすくできることを学びました。実は、**Claude に XMLタグを使って出力をより明確で人間にとって理解しやすくする**よう求めることもできます。

### 例

第2章でClaude に前置きを完全にスキップするよう求めることで解決した「詩の前置き問題」を覚えていますか？実は、**Claude に詩をXMLタグに入れるよう伝える**ことでも、同様の結果を得ることができます。

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

なぜこれをしたいのでしょうか？**XMLタグに出力があることで、エンドユーザーはXMLタグ間のコンテンツを抽出する短いプログラムを書くことで、詩だけを確実に取得できます**。

このテクニックの拡張は、**最初のXMLタグを`assistant`ターンに置くこと**です。`assistant`ターンにテキストを置くと、基本的にClaude がすでに何かを言ったことをClaude に伝え、そこから続けるべきだと指示することになります。このテクニックは「Claude の代弁」または「Claude の応答の事前入力」と呼ばれます。

下記では、最初の`<haiku>`XMLタグでこれを行いました。Claude が私たちが残した場所から直接続けることに注目してください。

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Claude は他の出力フォーマットスタイル、特に`JSON`の使用にも秀でています。JSON出力を強制したい場合（決定論的ではありませんが、それに近く）、Claude の応答を開始括弧`{`で事前入力することもできます。

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

下記は**同じプロンプト内の複数の入力変数と出力フォーマット仕様、すべてXMLタグを使用して行われている**例です。

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

#### ボーナスレッスン

API経由でClaude を呼び出している場合、閉じXMLタグを`stop_sequences`パラメータに渡すことで、Claude が希望するタグを出力したらサンプリングを停止させることができます。これにより、Claude が既にあなたが関心のある答えを与えた後の締めくくりのコメントを除去することで、お金と最終トークンまでの時間を節約できます。

上記のコンテンツを変更せずにレッスンのプロンプトを試したい場合は、レッスンノートブックの一番下までスクロールして[**サンプル練習場**](#example-playground)にアクセスしてください。

---

## 演習
- [演習 5.1 - Steph Curry GOAT](#exercise-51---steph-curry-goat)
- [演習 5.2 - 二つの俳句](#exercise-52---two-haikus)
- [演習 5.3 - 二つの俳句、二つの動物](#exercise-53---two-haikus-two-animals)

### 演習 5.1 - Steph Curry GOAT
選択を迫られたとき、Claude はMichael Jordanを史上最高のバスケットボール選手に指定します。Claude に他の誰かを選ばせることはできるでしょうか？

`PREFILL`変数を変更して、**Claude に史上最高のバスケットボール選手はStephen Curryであるという詳細な議論をさせてください**。この演習の焦点なので、`PREFILL`以外は変更しないようにしてください。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# Prefill for Claude's response
PREFILL = ""

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、下のセルを実行してください！

In [ ]:
from hints import exercise_5_1_hint; print(exercise_5_1_hint)

### 演習 5.2 - 二つの俳句
XMLタグを使用して下記の`PROMPT`を修正し、Claude が動物について一つではなく二つの俳句を書くようにしてください。一つの詩がどこで終わり、他方がどこで始まるかが明確でなければなりません。

In [ ]:
# Variable content
ANIMAL = "cats"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、下のセルを実行してください！

In [ ]:
from hints import exercise_5_2_hint; print(exercise_5_2_hint)

### 演習 5.3 - 二つの俳句、二つの動物
下記の`PROMPT`を修正して、**Claude が二つの異なる動物について二つの俳句を作成する**ようにしてください。最初の置換の代わりに`{ANIMAL1}`を、二番目の置換の代わりに`{ANIMAL2}`を使用してください。

In [ ]:
# First input variable
ANIMAL1 = "Cat"

# Second input variable
ANIMAL2 = "Dog"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL1}. Put it in <haiku> tags."

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、下のセルを実行してください！

In [ ]:
from hints import exercise_5_3_hint; print(exercise_5_3_hint)

### おめでとうございます！

ここまでのすべての演習を解いた場合、次の章に進む準備ができています。楽しいプロンプティングを！

---

## サンプル練習場

これは、このレッスンで示されたプロンプトの例を自由に試し、プロンプトを調整してClaude の応答にどのような影響を与えるかを確認するためのエリアです。

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))